In [5]:
import area_weight
import numpy as np
import xarray as xr

In [6]:
# Example resolution
lat_res = 1.0  # 1 degree
lon_res = 2.0  # 2 degrees

# Create 1D latitude and longitude arrays
lats = np.arange(-90, 90+lat_res, lat_res)
lons = np.arange(-180, 180+lon_res, lon_res)

# Create a dummy DataArray (just to illustrate xarray usage)
ds = xr.DataArray(
    data=np.random.rand(len(lats), len(lons)),
    dims=('lat', 'lon'),
    coords={'lat': lats, 'lon': lons},
    name='dummy_data'
)

# Slice Arctic region: lat from 60 to 90
ds_arctic = ds.sel(lat=slice(60, 90))

# Extract the lat/lon arrays we will work with
lat_arctic = ds_arctic.lat.values
lon_arctic = ds_arctic.lon.values


In [12]:
##############################################################################
# 3) Compute area weights using three methods
##############################################################################

# Put them into DataArrays for convenience
w_hav_da = area_weight.get_weights( area_weight.area_weight_haversine, lat_arctic, lon_arctic )

w_cos_da = area_weight.get_weights( area_weight.area_weight_cosine, lat_arctic, lon_arctic )

w_r_da = area_weight.get_weights( area_weight.area_weight_r_sinlat, lat_arctic, lon_arctic )


In [13]:

##############################################################################
# 4) Compare/inspect results
##############################################################################
# Example: Print some statistics or differences
print("Haversine-based weights:")
print(w_hav_da)

print("\nCosine-based weights:")
print(w_cos_da)

print("\nr^2 * delta(sin(lat)) * delta(lon) based weights:")
print(w_r_da)


# Optionally, you could plot them (requires matplotlib):
# diff_hav_cos.plot(x='lon', y='lat', robust=True)
# plt.show()



Haversine-based weights:
<xarray.DataArray 'weight' (lat: 31, lon: 181)> Size: 45kB
array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.        ],
       [0.9696185 , 0.9696185 , 0.9696185 , ..., 0.9696185 , 0.9696185 ,
        0.        ],
       [0.93894171, 0.93894171, 0.93894171, ..., 0.93894171, 0.93894171,
        0.        ],
       ...,
       [0.06979811, 0.06979811, 0.06979811, ..., 0.06979811, 0.06979811,
        0.        ],
       [0.03490437, 0.03490437, 0.03490437, ..., 0.03490437, 0.03490437,
        0.        ],
       [0.03490437, 0.03490437, 0.03490437, ..., 0.03490437, 0.03490437,
        0.03490437]])
Coordinates:
  * lat      (lat) float64 248B 60.0 61.0 62.0 63.0 64.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 1kB -180.0 -178.0 -176.0 -174.0 ... 176.0 178.0 180.0

Cosine-based weights:
<xarray.DataArray 'weight' (lat: 31, lon: 181)> Size: 45kB
array([[5.00000000e-01, 5.00000000e-01, 5.00000000e-01, ...,
        5.00000000e-0